# <center>Rapport de TP - implémentation de Hadoop MapReduce "from scratch" en Java"</center>

__Activer le kernel python3__

### <font color="red">Etape 0</font> : Mise en place de l'environnement

Avant de démarrer l'étude du mapreduce, nous allons télécharger les jeux de données nécessaires

In [1]:
# forestier mayotte
!wget https://raw.githubusercontent.com/legifrance/Les-codes-en-vigueur/master/forestier_mayotte.txt -O data/forestier_mayotte.txt

--2019-11-10 11:15:00--  https://raw.githubusercontent.com/legifrance/Les-codes-en-vigueur/master/forestier_mayotte.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.120.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1381 (1.3K) [text/plain]
Saving to: ‘data/forestier_mayotte.txt’

data/forestier_mayo 100%[===================>]   1.35K  --.-KB/s    in 0s      

2019-11-10 11:15:00 (8.74 MB/s) - ‘data/forestier_mayotte.txt’ saved [1381/1381]



In [1]:
# deontologie_police_nationale
!wget https://raw.githubusercontent.com/legifrance/Les-codes-en-vigueur/master/deontologie_police_nationale.txt -O data/deontologie_police_nationale.txt

--2019-11-10 11:16:02--  https://raw.githubusercontent.com/legifrance/Les-codes-en-vigueur/master/deontologie_police_nationale.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.120.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7488 (7.3K) [text/plain]
Saving to: ‘data/deontologie_police_nationale.txt’

data/deontologie_po 100%[===================>]   7.31K  --.-KB/s    in 0s      

2019-11-10 11:16:03 (38.0 MB/s) - ‘data/deontologie_police_nationale.txt’ saved [7488/7488]



In [2]:
# domaine plublic fluvial
!wget https://raw.githubusercontent.com/legifrance/Les-codes-en-vigueur/master/domaine_public_fluvial.txt -O data/domaine_public_fluvial.txt

--2019-11-10 11:16:04--  https://raw.githubusercontent.com/legifrance/Les-codes-en-vigueur/master/domaine_public_fluvial.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.120.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71173 (70K) [text/plain]
Saving to: ‘data/domaine_public_fluvial.txt’

data/domaine_public 100%[===================>]  69.50K  --.-KB/s    in 0.01s   

2019-11-10 11:16:04 (4.60 MB/s) - ‘data/domaine_public_fluvial.txt’ saved [71173/71173]



In [3]:
# sante publique
!wget https://raw.githubusercontent.com/legifrance/Les-codes-en-vigueur/master/sante_publique.txt -O data/sante_publique.txt

--2019-11-10 11:16:07--  https://raw.githubusercontent.com/legifrance/Les-codes-en-vigueur/master/sante_publique.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.120.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18111630 (17M) [text/plain]
Saving to: ‘data/sante_publique.txt’

data/sante_publique 100%[===================>]  17.27M  19.5MB/s    in 0.9s    

2019-11-10 11:16:09 (19.5 MB/s) - ‘data/sante_publique.txt’ saved [18111630/18111630]



In [8]:
!wget https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-13/segments/1490218189495.77/wet/CC-MAIN-20170322212949-00140-ip-10-233-31-227.ec2.internal.warc.wet.gz\
    -O data/big-file.gz
!cd data && gunzip big-file.gz

--2019-11-10 11:12:14--  https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-13/segments/1490218189495.77/wet/CC-MAIN-20170322212949-00140-ip-10-233-31-227.ec2.internal.warc.wet.gz
Resolving commoncrawl.s3.amazonaws.com (commoncrawl.s3.amazonaws.com)... 52.216.160.91
Connecting to commoncrawl.s3.amazonaws.com (commoncrawl.s3.amazonaws.com)|52.216.160.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154369177 (147M) [application/octet-stream]
Saving to: ‘data/big-file.gz’

data/big-file.gz    100%[===================>] 147.22M  17.7MB/s    in 9.2s    

2019-11-10 11:12:24 (16.0 MB/s) - ‘data/big-file.gz’ saved [154369177/154369177]



__Activer le kernel java__

### <font color="red">Etape 1</font> : Faire un programme séquentiel non parallélisé qui compte le nombre d'occurrences des mots dans un fichier. 

In [1]:
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.Paths;
import java.util.ArrayList;
import java.util.Collections;
import java.util.Comparator;
import java.util.HashMap;
import java.util.LinkedHashMap;
import java.util.Map;
import java.util.Map.Entry;

import static java.util.stream.Collectors.*;

public class WordsCount{
    
    public static ArrayList<String> read_file(String filename){
        ArrayList<String> lines = new ArrayList<>();
        try {
            lines = (ArrayList<String>) Files.readAllLines(Paths.get(filename));
        } catch (IOException e) {
            System.out.println("Problem when loading file");
            return null;
        }
        return lines;
    }
    
    public static ArrayList<String[]> tokenize(ArrayList<String> lines){
        ArrayList<String[]> tokenize_corpus = new ArrayList<>();
        for (String line : lines) {
            // removes punctuations
            line = line.replaceAll("\\p{Punct}","").toLowerCase().trim();

            tokenize_corpus.add(line.split(" "));
        }
        return tokenize_corpus;
    }
    
    public static HashMap<String, Double> words_count(String filename){
        // Function for the question 1
        ArrayList<String> lines = WordsCount.read_file(filename);
        ArrayList<String[]> lines_tokenized = WordsCount.tokenize(lines);
        HashMap<String, Double> words_count = new HashMap<>();

        for (String[] line_tokenized: lines_tokenized) {
            for (String word:line_tokenized) {
                if (word.isEmpty() != true){
                    if (words_count.get(word) == null) {
                            words_count.put(word, 1.0);
                    } else {
                            words_count.put(word, words_count.get(word)+1);
                    }
                }
            }
        }
        return words_count;
    }
    
    public static HashMap<String, Double> sorted_map_by_numeric_value(HashMap<String, Double> hash_map){
        // Function for question 2
        HashMap<String, Double> sorted_map = hash_map
                                            .entrySet()
                                            .stream()
                                            .sorted(Collections.reverseOrder(Map.Entry.comparingByValue()))
                                            .collect(toMap(Map.Entry::getKey, Map.Entry::getValue, (e1, e2) -> e2,
                                                    LinkedHashMap::new));
        return sorted_map;
    }
    
    public static HashMap<String, Double> sort(HashMap<String, Double> hash_map){
        // Function for question 3
        HashMap<String, Double> sorted_map = hash_map
                                                .entrySet()
                                                .stream()
                                                .sorted(new Comparator<Entry<String, Double>>() {
                                                    @Override
                                                    public int compare(Entry<String, Double> e1, Entry<String, Double> e2) {
                                                        if (e1.getValue().equals(e2.getValue())) {
                                                                return e1.getKey().compareTo(e2.getKey());
                                                        } else {
                                                            return e2.getValue().compareTo(e1.getValue());
                                                        }
                                                    }
                                                }).collect(toMap(Map.Entry::getKey, Map.Entry::getValue, (e1, e2) -> e1,
                                                        LinkedHashMap::new));
        return sorted_map;
    }
    
    public static void print_map(Map<String, Double> hash_map) {
        for (Entry<String, Double> el:hash_map.entrySet()) {
                System.out.println(el.getKey() + " " + el.getValue());
        }
    }
    
    public static void print_map(Map<String, Double> hash_map, Integer n) {
        Integer count = 0;
        for (Entry<String, Double> el:hash_map.entrySet()) {
            if(count == n){
                break;
            } else {
              System.out.println(el.getKey() + " " + el.getValue());
                count++;
            }
        }
    }
    
    public static void main(String[] args){
        System.out.print("Hello Jupyter from java");
    }
}

__1. Premier comptage en séquentiel pur__

Implémentez un logiciel en java qui compte le nombre d’occurrences des mots d’un fichier d’entrée de manière non parallélisée (monothread, une seul thread), en utilisant un seul processeur.<br> 
Quelle structure de donnée est la plus pertinente pour stocker les résultats: List, HashMap ou HashSet ou une autre ? Pour quelle raison ? 

 

Testez votre programme avec un fichier d’entrée input.txt avec comme contenu: 
```sh 
Deer Beer River 
Car Car River 
Deer Car Beer
```

Résultat: 
```sh 
Deer 2 
Beer 2 
River 2 
Car 3
```

In [21]:
WordsCount.print_map(WordsCount.words_count("data/input.txt"));

deer 2.0
car 3.0
river 2.0
beer 2.0


__2. Premier tri en séquentiel pur__

Modifiez votre programme pour trier par nombre d'occurrences: 

Résultat: 
```sh
Car 3 
Deer 2 
Beer 2 
River 2
```

In [17]:
HashMap<String, Double> words_count = WordsCount.words_count("data/input.txt");
WordsCount.print_map(WordsCount.sorted_map_by_numeric_value(words_count));

car 3.0
deer 2.0
river 2.0
beer 2.0


__3. Deuxième tri alphabétique en séquentiel pur__

Modifiez le programme pour trier alphabétiquement pour les mots à égalité du nombre d’occurrences: 

Résultat: 
```sh
Car 3 
Beer 2 
Deer 2
River 2 
```

In [22]:
HashMap<String, Double> words_count = WordsCount.words_count("data/input.txt");
WordsCount.print_map(WordsCount.sort(words_count));

car 3.0
beer 2.0
deer 2.0
river 2.0


__4. Test du programme séquentiel sur le code forestier de Mayotte__

Testez ensuite votre programme avec le code forestier de Mayotte disponible sur github forestier_mayotte.txt :  
https://github.com/legifrance/Les-codes-en-vigueur 

Votre programme a-t-il fonctionné du premier coup ?

Vérifiez en ouvrant le fichier texte qu’il contient bien du texte et non du code HTML. 

Ne perdez pas de temps à corriger les éventuelles erreurs dues aux caractères spéciaux ou à des mots suspects ou illisibles (de toutes façons par la suite il y aura du chinois dans le texte).

Le programme à fonctionné du premier coup, cependant les caractères spéciaux et la ponctuation faussaient les résultats. Comme demandé dans l'énoncé, je n'ai pas passé beaucoup de temps sur le nettoyage des données. J'ai seulement ajouté une regex spéciale Java afin de supprimer la ponctuation.

In [3]:
HashMap<String, Double> words_count = WordsCount.words_count("data/forestier_mayotte.txt");
WordsCount.print_map(WordsCount.sort(words_count), 15);

de 12.0
biens 8.0
ou 8.0
code 6.0
forestier 6.0
des 5.0
partie 5.0
agroforestiers 4.0
aux 4.0
communes 4.0
dispositions 4.0
forestiers 4.0
gestion 4.0
le 4.0
les 4.0


__5. Les 50 mots du code de la déontologie de la police nationale__

Testez votre programme avec le code de déontologie de la police nationale disponible sur github deontologie_police_nationale.txt : https://github.com/legifrance/Les-codes-en-vigueur 

De même ne perdez pas de temps à filtrer les caractères spéciaux ou autres mots bizarres. Pourquoi ? Car nous travaillerons ensuite sur des textes en chinois, japonais, arabe et d’autres langues. Si vous implémentez une étape de filtrage ici en français elle ne servira à rien par la suite. Quels sont les 5 premiers mots (qui ressemblent à des mots) parmi les 50 premiers de la liste triée résultat ? Gardez la réponse pour l’intégrer au rapport.

Les 5 premiers mots parmi les 50 premiers de la liste triée sont "de, la, police, et, des"

In [4]:
HashMap<String, Double> words_count = WordsCount.words_count("data/deontologie_police_nationale.txt");
WordsCount.print_map(WordsCount.sort(words_count), 50);

de 98.0
la 51.0
police 38.0
et 36.0
des 33.0
le 27.0
à 25.0
les 24.0
article 20.0
nationale 20.0
↬ 19.0
en 13.0
est 13.0
titre 13.0
ou 12.0
qui 11.0
fonctionnaires 10.0
lautorité 10.0
aux 9.0
code 9.0
fonctionnaire 9.0
par 9.0
commandement 8.0
du 8.0
leur 8.0
ses 8.0
au 7.0
devoirs 7.0
déontologie 7.0
il 7.0
ne 7.0
a 6.0
dans 6.0
cas 5.0
faire 5.0
lordre 5.0
ordres 5.0
pour 5.0
sa 5.0
se 5.0
si 5.0
sont 5.0
tout 5.0
doit 4.0
droits 4.0
elle 4.0
exécution 4.0
missions 4.0
pas 4.0
personne 4.0


__6. Les 50 mots du code du domaine public fluvial__

Testez votre programme avec le code du domaine public fluvial domaine_public_fluvial.txt. 

Quels sont les 5 premiers mots (qui ressemblent à des mots) parmi les 50 premiers de la liste triée résultat ?  Gardez la réponse pour l’intégrer au rapport.

Les 5 premiers mots parmi les 50 premiers de la liste triée sont "de, le, la, du, et"

In [5]:
HashMap<String, Double> words_count = WordsCount.words_count("data/domaine_public_fluvial.txt");
WordsCount.print_map(WordsCount.sort(words_count), 50);

de 630.0
le 429.0
la 370.0
du 347.0
et 295.0
les 240.0
des 222.0
à 209.0
est 173.0
dans 150.0
par 124.0
sur 123.0
ou 120.0
en 109.0
article 107.0
bateau 106.0
↬ 103.0
au 97.0
un 79.0
pour 73.0
tribunal 72.0
lieu 69.0
larticle 68.0
aux 66.0
il 66.0
dimmatriculation 64.0
qui 54.0
bureau 50.0
code 48.0
titre 48.0
navigation 46.0
que 46.0
dun 44.0
bateaux 43.0
où 43.0
son 43.0
domicile 42.0
se 41.0
créanciers 40.0
juge 38.0
a 37.0
propriétaire 37.0
commerce 36.0
saisie 36.0
intérieure 35.0
délai 34.0
nom 34.0
prix 34.0
sil 33.0
cas 32.0


__7. Les 50 mots du code de la santé publique__

Testez votre programme avec le code de la santé publique sante_publique.txt. 

Quels sont les 5 premiers mots (qui ressemblent à des mots) parmi les 50 premiers de la liste triée résultat ?  Gardez la réponse pour l’intégrer au rapport. 

Les 5 premiers mots parmi les 50 premiers de la liste triée sont "de, le, des, à, et"

In [6]:
HashMap<String, Double> words_count = WordsCount.words_count("data/sante_publique.txt");
WordsCount.print_map(WordsCount.sort(words_count), 50);

de 190889.0
la 82599.0
des 67813.0
à 65546.0
et 62702.0
les 62474.0
le 56800.0
du 48514.0
ou 40848.0
en 31742.0
par 30718.0
au 25730.0
dans 25104.0
article 23844.0
larticle 22998.0
↬ 21810.0
est 21792.0
un 20536.0
santé 20197.0
l 18762.0
pour 16876.0
aux 16144.0
sont 15048.0
une 13868.0
sur 12624.0
que 11348.0
r 11236.0
qui 10150.0
dun 9672.0
peut 8608.0
directeur 8604.0
lagence 8284.0
conditions 8094.0
être 8070.0
cas 8040.0
ne 7714.0
a 7622.0
conseil 7560.0
dune 7374.0
son 7298.0
général 6912.0
il 6666.0
ce 6352.0
ces 6188.0
1° 6114.0
dispositions 5926.0
2° 5872.0
leur 5766.0
pas 5702.0
sécurité 5282.0


__8. Chronométrage séquentiel__

Chronométrer votre programme sur le code de la santé publique.

In [7]:
double start_time = System.currentTimeMillis();
HashMap<String, Double> words_count = WordsCount.words_count("data/sante_publique.txt");
double end_time = System.currentTimeMillis();
double total_time = end_time - start_time;
System.out.print(String.format("Durée du word count: %f s", total_time/1000))

Durée du word count: 1.012000 s

__9. Travailler sur des gros fichiers__

Testez votre programme sur un cas réel: un extrait de toutes les pages internet transformées au format texte brut (format WET). Toutes les pages sur  internet au format texte sont disponibles sur [commoncrawl](http://commoncrawl.org/the-data/get-started/) : chaque mois, environ 3 milliards de pages web, soit 250 To de données sont stockées. Ces données sont disponibles par tranche de moins d’1Go environ, vous travaillerez sur une tranche de 380Mo.
J’ai choisi une tranche en particulier pour avoir une comparaison entre nous (vous pouvez tester sur d’autres tranches si vous voulez). Téléchargez cette tranche ici:
[gros fichier](https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-13/segments/1490218189495.77/wet/CC-MAIN-20170322212949-00140-ip-10-233-31-227.ec2.internal.warc.wet.gz) 
Décompressez et obtenez le fichier CC-MAIN-20170322212949-00140-ip-10-233-31-227.ec2.internal.warc.wet 
Il s’agit d’une tranche contenant un ensemble de sites internet au format texte brut (WET).
Testez votre programme avec ce fichier en entrée. Chronométrez-le.

In [2]:
double start_time = System.currentTimeMillis();
HashMap<String, Double> words_count = WordsCount.words_count("data/big-file");
double end_time = System.currentTimeMillis();
double total_time = end_time - start_time;
System.out.print(String.format("Durée du word count: %f s", total_time/1000))

Durée du word count: 58.340000 s

__Activer le kernel phython3__

### <font color="red">Etape 2</font> : Travailler avec plusieurs ordinateurs en réseau.

__1. Nom court, nom long__

Quel est le nom COURT de votre ordinateur (le nom simple sans le domaine) ? Quel est le nom LONG de votre ordinateur (le nom avec le domaine) ? Comment les connaître en ligne de commande ? Sur les ordinateurs de l’école, est-il possible d’obtenir ces noms autrement qu’en ligne de commande ? Ajoutez les réponses à votre rapport.

Dans l'architecture docker actuelle, il n'y a pas de différence entre le nom court et le nom long des machines

In [4]:
!echo Nom long: `hostname -f`
!echo Nom court: `hostname`

Nom long: master
Nom court: master


__2. Adresse ip__

Comment connaître les adresses (plusieurs) IP de votre ordinateur en ligne de commande ? Autrement (en passant par un site internet par exemple) ? Ajoutez les réponses à votre rapport.

In [5]:
!echo Adresse ip : `hostname -I`

Adresse ip : 192.168.208.4


__3. Du nom vers l’IP__

Comment à partir du nom d’un ordinateur, obtenir les adresses IP en ligne de commande ? Ajoutez les réponses à votre rapport.

In [1]:
!host worker-1

worker-1 has address 172.19.0.4


__4. De l’IP vers le nom__

Comment, à partir d’une adresse IP, obtenir les noms associés en ligne de commande ? Ajoutez les réponses à votre rapport.

In [2]:
!host 172.19.0.4

4.0.19.172.in-addr.arpa domain name pointer worker-1.docker_default.


__5. Ping pong à l’intérieur!__

Testez la communication avec d’autres ordinateurs (pas le vôtre) depuis le réseau de l’école en utilisant la commande ping (pour arrêter le ping faire CTRL + C). suivi du nom court, du nom long, de l’IP. Les trois méthodes fonctionnent-elles ? Ajoutez les réponses à votre rapport.

In [3]:
!ping worker-1

PING worker-1 (172.19.0.4) 56(84) bytes of data.
64 bytes from worker-1.docker_default (172.19.0.4): icmp_seq=1 ttl=64 time=0.075 ms
64 bytes from worker-1.docker_default (172.19.0.4): icmp_seq=2 ttl=64 time=0.088 ms
64 bytes from worker-1.docker_default (172.19.0.4): icmp_seq=3 ttl=64 time=0.058 ms
^C

--- worker-1 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2056ms
rtt min/avg/max/mdev = 0.058/0.073/0.088/0.015 ms


In [4]:
!ping 172.19.0.4

PING 172.19.0.4 (172.19.0.4) 56(84) bytes of data.
64 bytes from 172.19.0.4: icmp_seq=1 ttl=64 time=0.088 ms
64 bytes from 172.19.0.4: icmp_seq=2 ttl=64 time=0.130 ms
64 bytes from 172.19.0.4: icmp_seq=3 ttl=64 time=0.083 ms
^C

--- 172.19.0.4 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2061ms
rtt min/avg/max/mdev = 0.083/0.100/0.130/0.022 ms


Toutes les méthodes fonctionnent

__7. Calculer en ligne de commande sur l’ordinateur local__

Comment lancer un calcul en ligne de commande sur votre ordinateur (par exemple 2 + 3) ? Parmi les multiples réponses possibles, lesquelles permettent de lancer le calcul et d’obtenir le résultat en appuyant une seule fois sur la touche <Entrée> ? Ajoutez les réponses à votre rapport.

In [5]:
!echo $((3+5)) 

8


__8. Calculer en ligne de commande sur un ordinateur distant__

Comment lancer un calcul  (par exemple 2 + 3) en ligne de commande sur un autre ordinateur (à distance) ? Il faudra certainement vous authentifier avec un mot de passe. Comment obtenir le résultat du calcul immédiatement après avoir tapé son mot de passe ? Ajoutez les réponses à votre rapport.

La communication ssh sans mot de passe entre les containers docker est déjà configurée.

In [8]:
!ssh root@worker-1 echo $((3+5))

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
8


### <font color="red">Etape 3</font> : Travailler avec des fichiers locaux ou sur un serveur NFS..

__1. Chemin absolu__

Quel est le chemin absolu de votre répertoire personnel, votre home directory ? (commandes “cd” puis ”pwd”)

In [9]:
!cd && pwd

/home/jovyan


__2. Un fichier dans le répertoire personnel__

Créez un fichier fperso.txt contenant le texte “bonjour” dans votre répertoire personnel (sur un ordinateur de l’école). 
Vérifiez le contenu du fichier avec cette commande exactement:
```sh
cat ~/fperso.txt
```

In [12]:
!echo bonjour > $HOME/fperso.txt && cat $HOME/fperso.txt

bonjour


__3. Ou se trouve le fichier dans le répertoire personnel__

Ce fichier est-il sur le disque dur de l’ordinateur ou autre part ? Comment savoir où est stocké physiquement ce fichier, à l’aide de quelle commande ?

In [15]:
!df -k $HOME/fperso.txt

Filesystem     1K-blocks     Used Available Use% Mounted on
overlay        263174212 25063008 224673048  11% /


__4. Un dossier et un fichier dans le répertoire temporaire__

Créez un dossier `/tmp/<votre nom d’utilisateur>` en remplaçant <votre nom d’utilisateur> (ne pas mettre les caractères < et > ).
Créez un fichier ftemp.txt dans le répertoire `/tmp/<votre nom d’utilisateur>` .
Vérifiez le contenu du fichier avec cette commande exactement:
```sh
cat /tmp/<votre nom d’utilisateur>/ftemp.txt
```
Ce dossier et ce fichier sont-ils sur le disque dur de l’ordinateur ou autre part ? Comment savoir où sont stockés physiquement ces éléments, à l’aide de quelle commande ?

In [6]:
!mkdir /tmp/$NB_USER && echo bonjour > /tmp/$NB_USER/ftemp.txt
!cat /tmp/$NB_USER/ftemp.txt

# To know where the file is stored
!df -k /tmp/$NB_USER/ftemp.txt

bonjour
Filesystem     1K-blocks     Used Available Use% Mounted on
overlay        263174212 25515584 224220472  11% /


__Copie de ficher entre ordinateur distant__

In [7]:
!ssh root@worker-1 mkdir /tmp/$NB_USER
!scp /tmp/$NB_USER/ftemp.txt root@worker-1:/tmp/$NB_USER
!ssh root@worker-1 ls /tmp/$NB_USER

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
mkdir: cannot create directory '/tmp/jovyan': File exists
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
ftemp.txt                                     100%    8     5.6KB/s   00:00    
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
ftemp.txt


### <font color="red">Etape 4</font> : Lancer des programmes java à distance manuellement.

__Activer le kernel java__

__Un premier programme SLAVE sous Eclipse__

Faire un programme Java nommé “SLAVE” qui calcule 3+5, affiche le résultat, sous Eclipse (Pour lancer Eclipse: Menu applications>développement), lancer ce programme dans Eclipse (flèche verte “exécuter”)

In [1]:
public class Slave {

    public static void main() throws InterruptedException {
        System.out.println(3 + 5);
    }
}

In [2]:
new Slave().main()

8


__1. Exportation en JAR__

Exporter le programme Java en slave.jar exécutable (Java ARchive dite Runnable) sous Eclipse. Attention de bien vérifier que le JAR est de type “Runnable”/”exécutable”.

Le fichier `slave.jar` se trouve dans dossier `hadoop-from-scratch/jar`

__2. Exécution sur disque dur local__

Créez le répertoire `/tmp/<votre nom d’utilisateur>/`
Copiez  slave.jar exécutable dans le répertoire `/tmp/<votre nom d’utilisateur>/`
Testez et Lancer le slave.jar en ligne de commande sur votre ordinateur local.

__Activer le kernel python3__

In [1]:
!cp jar/slave.jar /tmp/$NB_USER && java -jar /tmp/$NB_USER/slave.jar

8


__3. Copie du JAR et exécution distante__

Depuis la machine A contenant `/tmp/<votre nom d’utilisateur>/slave.jar` 
Créez à distance sur la machine B (s’il n’existe pas) un répertoire `/tmp/<votre nom d’utilisateur>/`
Copiez slave.jar sur la machine B dans le répertoire `/tmp/<votre nom d’utilisateur>/`
Exécutez à distance (depuis A sur la machine B) le slave.jar.
Quelle est la commande tapée pour effectuer cette dernière action ?

In [4]:
!ssh root@worker-1 mkdir /tmp/$NB_USER
!scp jar/slave.jar root@worker-1:/tmp/$NB_USER
!ssh root@worker-1 java -jar /tmp/$NB_USER/slave.jar

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
mkdir: cannot create directory '/tmp/jovyan': File exists
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
slave.jar                                     100%   10KB  12.3MB/s   00:00    
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
8


__Activer le kernel java__

### <font color="red">Etape 5</font> : Lancer des programmes en ligne de commande depuis java et afficher la sortie standard et la sortie d’erreur.

__1. Un programme MASTER java qui lance un autre programme en ligne de commande!__

Ecrire un programme java nommé “MASTER” qui lance la commande suivante en utilisant ProcessBuilder:
```sh
ls -al /tmp
```
(vous pouvez aussi tester cette commande dans un terminal avant)
Récupérer et afficher la sortie de cette commande.
Vous devez utiliser ProcessBuilder de cette façon:

```java
ProcessBuilder pb = new ProcessBuilder("ls", “-al”, “/tmp”);
```

In [18]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;

public class Master {
    public static void run_shell_command(String[] command) {
        ProcessBuilder pb = new ProcessBuilder(command);

        try {
            Process process = pb.start();
            BufferedReader reader_input = new BufferedReader(new InputStreamReader(process.getInputStream()));

            String line;
            while ((line = reader_input.readLine()) != null) {
                System.out.println(line);
            }

        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    public static void main(String[] args) throws IOException {
        Master.run_shell_command(args[0].split(" "));
    }
}

In [20]:
new Master().main(new String[]{"ls -al /tmp"});

total 648
drwxrwxrwt 17 root root   4096 Oct 13 16:39 .
drwxr-xr-x 23 root root   4096 Oct 10 13:26 ..
drwxrwxr-x  2 axel axel   4096 Oct 13 11:59 acamara
-rw-rw-r--  1 axel axel    405 Oct  7 23:15 gradle-worker-classpath1054293946014425259txt
-rw-r--r--  1 1000 1000    405 Sep 27 16:32 gradle-worker-classpath10796728312708717652txt
-rw-r--r--  1 1000 1000    405 Sep 27 17:56 gradle-worker-classpath16526685033490323404txt
-rw-r--r--  1 1000 1000    405 Sep 27 16:28 gradle-worker-classpath17730799258997419394txt
drwxr-xr-x  2 1000 1000   4096 Oct  3 15:42 hsperfdata_aksl
drwxr-xr-x  2 axel axel   4096 Oct 13 16:32 hsperfdata_axel
drwxr-xr-x  2 root root   4096 Sep 27 15:28 hsperfdata_root
-rw-rw-r--  1 axel axel   7273 Oct 12 16:06 kotlin-daemon.2019-10-12.14-52-46-710.00.log
-rw-rw-r--  1 axel axel   1328 Oct 12 16:56 kotlin-daemon.2019-10-12.16-56-18-013.00.log
-rw-rw-r--  1 axel axel   3451 Oct 12 17:08 kotlin-daemon.2019-10-12.16-59-57-526.00.log
-rw-rw-r--  1 axel axel  27300 Oct 

__2. Un programme MASTER java qui gère les erreurs de lancement d’un autre programme en ligne de commande.__

Modifiez votre programme”MASTER” pour qu’il affiche la sortie d’erreur en cas d’erreur lors de l’exécution de la commande. Testez la sortie d’erreur avec une commande qui échoue avec un sortie d’erreur. Essayez par exemple d’exécuter la commande “ls /jesuisunhero”.
Explications: si on tape la commande “ls /jesuiunhero”, le dossier /jesuisunhero n’existant pas, on aura une erreur de type “impossible d’accéder à /jesuisunhero: aucun fichier ou dossier de ce type.” qui s’affiche dans la sortie d’erreur. En effet, il y a deux sorties: les sorties standards (sans erreur) et les sorties d’erreurs.
Vous devez utiliser ProcessBuilder de cette façon:
```java
ProcessBuilder pb = new ProcessBuilder("ls", "/jesuisunhero”);
```
Sur `pb`, vous pouvez récupérer le flux de la sortie standard et le flux de la sortie d’erreur.

Afin de gérer les éventuelles erreurs lors du lancement du processus, nous pouvons utiliser la méthode `getErrorStream` de l'objet `Process`. Cette méthode revoie le flux de messages d'erreur du processus. 

In [1]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;

public class Master {
    public static void run_shell_command(String[] command) {
        ProcessBuilder pb = new ProcessBuilder(command);

        try {
            Process process = pb.start();
            BufferedReader reader_input = new BufferedReader(new InputStreamReader(process.getInputStream()));

            String line;
            while ((line = reader_input.readLine()) != null) {
                System.out.println(line);
            }
            
            BufferedReader reader_error = new BufferedReader(new InputStreamReader(process.getErrorStream()));

            while ((line = reader_error.readLine()) != null) {
                System.out.println(line);
            }

        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    public static void main(String[] args) throws IOException {
        Master.run_shell_command(args[0].split(" "));
    }
}

In [2]:
new Master().main(new String[]{"ls -al /fr"});

ls: cannot access '/fr': No such file or directory


__3. Un programme MASTER java qui lance un slave.jar en ligne de commande.__

Modifiez votre programme “MASTER” pour qu’il lance “SLAVE”, c’est à dire slave.jar situé sur la même machine que “MASTER” dans le dossier 
```sh
/tmp/<votre nom d’utilisateur>/slave.jar
```

In [4]:
new Master().main(new String[]{"java -jar jar/slave.jar"});

8


### <font color="red">Etape 6</font> : Gérer les timeout du MASTER.

__1. Un SLAVE qui simule un calcul de 10 secondes.__

Modifiez votre programme SLAVE pour qu’il simule une attente de 10 secondes avant d’afficher le résultat du calcul 3+5. Pour cela utilisez
`Thread.sleep(10000);` 
Vérifiez le bon fonctionnement du SLAVE et constatez qu’il y a 10 secondes entre le démarrage du SLAVE et l’affichage du résultat. Attention de ne rien afficher avant les 10 secondes pour que la question suivante fonctionne correctement.
Générez de nouveau le slave.jar. Copiez-le en écrasant le slave.jar dans le dossier `/tmp/<votre nom d’utilisateur>/slave.jar`
Testez le lancement à partir de MASTER.

In [2]:
public class Slave {

    public static void main() throws InterruptedException {
        Thread.sleep(10000);
        System.out.println(3 + 5);
    }
}

In [3]:
new Slave().main()

8


__2. Gérer les timeout au niveau du MASTER__

Modifier le MASTER pour qu’il attende que quelque chose soit écrit dans la sortie standard (sans erreur) ou dans la sortie d’erreurs du SLAVE pendant un certain temps maximum. Au bout du temps imparti le MASTER considère un timeout.
Il arrête les éventuels threads (si vous utilisez des threads - non obligatoire) s’occupant des sorties et/ou stoppe le processus créé avec ProcessBuilder.

Vous devrez vérifier les TESTs suivants:
- TEST1 : Testez le bon fonctionnement du timeout en lançant le SLAVE avec un timeout de 2 secondes sur les sorties (standard et d’erreur). Le timeout étant plus court (au niveau du MASTER 2 secondes) que le temps de calcul du SLAVE (10 secondes), le MASTER doit arrêter les éventuels threads (si vous en utilisez) et le processus.
- TEST 2: Testez ensuite avec un timeout de 15 secondes, il ne devrait pas y avoir de timeout.
- TEST 3: Testez ensuite en changeant le SLAVE pour qu’il écrive non plus dans la sortie standard (sans erreur) mais dans la sortie d’erreur. Pour cela, remplacez dans le Slave les System.out.print… par System.err.print…

In [ ]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStream;
import java.io.InputStreamReader;
import java.util.Arrays;
import java.util.concurrent.LinkedBlockingQueue;
import java.util.concurrent.TimeUnit;

public class Master {

    static class ProcessLauncher {
        Process process;
        Integer timeout;
        String[] command;
        boolean error = false;
        ThreadReaderStream input_stream;
        ThreadReaderStream error_stream;

        ProcessLauncher(String command, Integer timeout) {
            this.timeout = timeout;
            this.command = command.split(" ");
            ProcessBuilder builder = new ProcessBuilder(this.command);
            try {
                this.process = builder.start();
            } catch (IOException e1) {
                e1.printStackTrace();
            }
            assert process != null;
            input_stream = new ThreadReaderStream(process.getInputStream());
            error_stream = new ThreadReaderStream(process.getErrorStream());
            input_stream.start();
            error_stream.start();
        }

        boolean launch_process() throws InterruptedException {
            String line;
            while (((line = input_stream.queue.poll(this.timeout, TimeUnit.SECONDS)) != null)) {
                System.out.print(line + "\n");
            }
            while (((line = error_stream.queue.poll()) != null)) {
                System.out.print(line + "\n");
                error = true;
            }
            return !error;
        }
    }

    static class ThreadReaderStream extends Thread {
        LinkedBlockingQueue<String> queue = new LinkedBlockingQueue<>();
        BufferedReader reader;

        ThreadReaderStream(InputStream stream) {
            this.reader = new BufferedReader(new InputStreamReader(stream));
        }

        @Override
        public void run() {
            try {
                String line;
                while ((line = reader.readLine()) != null) {
                    queue.put(line);
                }
            } catch (IOException | InterruptedException e) {
                e.printStackTrace();
            }
        }
    }


    public static void main(String[] args) throws InterruptedException {
        // Deploy the
        String[] pc = args;
        // Une facon de lancer les threads, sans attendre de retour de leur part:
        Arrays.asList(pc).parallelStream()
                .forEach(command -> {
                    try {
                        new ProcessLauncher(command, 2).launch_process();
                    } catch (InterruptedException e) {
                        e.printStackTrace();
                    }});
    }
}

### <font color="red">Etape 7</font> : Déployer automatiquement le programme SLAVE sur un ensemble de machines.

A partir de cette étape, les classes présentes dans le package `src.mapreduce` du projet seront utilisées pour répondre aux questions. Afin de simplifier le rapport, certaines questions seront traitées brièvement. Le package `mapreduce` coontient 5 classes :

- Master : C'est le programme master du mapreduce.
- Slave : Correspond au code exécuté sur les workers (map, shuffle et reduce).
- Deploy : Utilisée pour déployer des éléments sur les workers
- Clean : Utilisée pour nettoyer les environnements sur les workers
- Utils : Contient les fonctions utiles communes entre toutes les classes

Tous les lancements de processus sur les machines sont fait en parallèles via les méthodes de la classe `Utils` :

- `launch_actions_with_return` : Lance un processus et récupère la sortie.
- `launch_actions_without_return` : Lance un processus sans attendre de retour.

La méthode `parallelStream()` permet de parcourir et de lancer chaque commande de notre liste en parallèle.

In [21]:
import java.io.*;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;
import java.util.ArrayList;
import java.util.Collections;
import java.util.List;
import java.util.concurrent.LinkedBlockingQueue;
import java.util.concurrent.TimeUnit;
import java.util.stream.Collectors;
import java.util.stream.Stream;

class Utils {


    static ArrayList<String> read_file(String filename) {
        ArrayList<String> lines;
        try {
            lines = (ArrayList<String>) Files.readAllLines(Paths.get(filename));
        } catch (IOException e) {
            System.out.println("Problem when loading file");
            return null;
        }
        return lines;
    }

    static ArrayList<String[]> tokenize(ArrayList<String> lines) {
        ArrayList<String[]> tokenize_corpus = new ArrayList<>();
        for (String line : lines) {
            // removes punctuations
            line = line.toLowerCase().trim();

            tokenize_corpus.add(line.split(" "));
        }

        return tokenize_corpus;
    }

    static void launch_actions_without_return(ArrayList<String> actions) throws InterruptedException {
        ArrayList<ProcessLauncher> launchers = new ArrayList<>();
        for (String command : actions) {
            launchers.add(new ProcessLauncher(command, 2));
            System.out.println(command);
        }

        launchers.parallelStream().forEach(ProcessLauncher::launch_process);

        for (ProcessLauncher launcher : launchers)
            launcher.input_stream.join();
    }

    static List<Boolean> launch_actions_with_return(ArrayList<String> actions) throws InterruptedException {
        ArrayList<ProcessLauncher> launchers = new ArrayList<>();
        for (String command : actions) {
            launchers.add(new ProcessLauncher(command, 2));
        }
        List<Boolean> returnValue = launchers.parallelStream()
                .map(ProcessLauncher::launch_process).collect(Collectors.toCollection(ArrayList::new));
        for (ProcessLauncher launcher : launchers)
            launcher.input_stream.join();
        return returnValue;
    }

    static List<String> list_directory(String path_to_directory){
        try (Stream<Path> walk = Files.walk(Paths.get(path_to_directory))) {

            return walk.filter(Files::isRegularFile)
                    .map(Path::toString).collect(Collectors.toList());
        } catch (IOException e) {
            e.printStackTrace();
        }
        return Collections.emptyList();
    }

    static void write_file(List<String> word_count, String filename, String mode) throws IOException {
        if (mode.equals("a")){
            try{
                FileWriter fstream = new FileWriter(filename,true);
                BufferedWriter writer = new BufferedWriter(fstream);
                for (String line:word_count){
                    writer.write(line + "\n");
                }
                writer.close();
            }catch (Exception e){
                System.err.println("Error while writing to file: " +
                        e.getMessage());
            }
        } else if (mode.equals("w")){
            BufferedWriter writer = new BufferedWriter(new FileWriter(filename));
            for (String line:word_count){
                writer.write(line + "\n");
            }
            writer.close();
        }
    }

    static class ProcessLauncher {
        Process process;
        Integer timeout;
        String[] command;
        ThreadReaderStream input_stream;
        ThreadReaderStream error_stream;

        ProcessLauncher(String command, Integer timeout) {
            this.timeout = timeout;
            this.command = command.split(" ");
            ProcessBuilder builder = new ProcessBuilder(this.command);
            try {
                this.process = builder.start();
            } catch (IOException e1) {
                e1.printStackTrace();
            }
            assert process != null;
            input_stream = new ThreadReaderStream(process.getInputStream());
            error_stream = new ThreadReaderStream(process.getErrorStream());
            input_stream.start();
            error_stream.start();
        }

        boolean launch_process() {
            String line;
            boolean running = true;
            boolean tooLong = false;
            while (running) {
                try {
                    // Wait For retourne vrai si le programme est arrete
                    boolean stillRunning = !process.waitFor(5, TimeUnit.SECONDS);
                    // On lit la sortie standard. Si on a eu quelque chose, on continue
                    // ​
                    if (!input_stream.queue.isEmpty()) {
                        // On a du monde dans le buffer. On les recupere.
                        // Si on ne veut pas les récuperer, on peut faire un "reset"
                        // reader.reset();
                        while (((line = input_stream.queue.poll()) != null)) {
                            System.out.println(line);
                        }
                    } else if (!error_stream.queue.isEmpty()) {
                        // On a du monde dans le buffer. On les recupere.
                        // Si on ne veut pas les récuperer, on peut faire un "reset"
                        // reader.reset();
                        while (((line = error_stream.queue.poll()) != null)) {
                            System.out.println(line);
                        }
                    } else if (stillRunning) {
                        // Le process n'a rien écris pendant les 5 secondes. On le tue
                        tooLong = true;
                        process.destroy();
                    }
                    running = stillRunning && !tooLong;
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }
            return !tooLong && process.exitValue()==0;
        }
    }

    static class ThreadReaderStream extends Thread {
        LinkedBlockingQueue<String> queue = new LinkedBlockingQueue<>();
        BufferedReader reader;

        ThreadReaderStream(InputStream stream) {
            this.reader = new BufferedReader(new InputStreamReader(stream));
        }

        @Override
        public void run() {
            try {
                String line;
                while ((line = reader.readLine()) != null) {
                    queue.put(line);
                }
            } catch (IOException | InterruptedException e) {
                e.printStackTrace();
            }
        }
    }
}

__1. Un programme DEPLOY : Test de connection SSH multiple__

Créer un fichier texte à la main contenant : les adresses IP et/ou les noms des machines que nous voulons utiliser pour notre système réparti (par exemple toutes les machines de cette salle de TP), avec un nom ou une IP par ligne dans le fichier.
Créer un nouveau programme java DEPLOY qui lit ce fichier ligne par ligne et teste si la connection SSH fonctionne bien sur chacune des machines. Attention, il s’agit bien d’un nouveau programme qui est séparé de MASTER ou SLAVE, vous ne l’exécuterez qu’en cas de mise à jour du SLAVE. Ceci permet de vérifier qu’une machine n’est pas éteinte ou qu’il y a un problème de connection (par exemple).
Pour vérifier que la connection fonctionne bien, vous pouvez faire afficher le nom de la machine distante (en exécutant la commande hostname à distance) et vérifier que l’affichage a effectivement lieu, sans erreurs. Réutilisez des parties de codes de la cinquième étape.

Votre programme DEPLOY lance-t-il les connections de manière séquentielle (les unes après les autres) ou de manière parallèle?


__2. Un programme DEPLOY : copie de slave.jar multiple__

Modifier votre programme DEPLOY pour qu’il copie le slave.jar dans `/tmp/<votre nom d’utilisateur>/` sur les ordinateurs dont la connection SSH  fonctionne.

Pour cela, vous devez utiliser la commande mkdir -p pour créer les répertoires dans /tmp s’ils n’existent pas déjà, attendre que le mkdir se termine puis copier avec scp le fichier slave.jar. Comment faites-vous pour attendre que le mkdir se termine correctement?
Vérifiez ensuite manuellement que le fichier a bien été copié sur les ordinateurs distants.
Attention de bien attendre la fin du mkdir avant de lancer le scp (on ne veut pas avoir de copie avant que le dossier soit effectivement créé).

Lors des copies, faites attention au caractère “ / ” à la fin d’un chemin :
 /tmp/toto est un chemin vers un fichier nommé toto
/tmp/toto/ est un chemin vers un dossier nommé toto.

Votre programme DEPLOY lance-t-il les copies de manière séquentielle (les unes après les autres) ou de manière parallèle?

Votre programme DEPLOY lance-t-il les copies de manière séquentielle (les unes après les autres) ou de manière parallèle?

In [2]:
import java.util.*;

public class Deploy {

    public static Boolean deploy(List<String> hostnames, String host_path, String remote_path) throws InterruptedException {
        ArrayList<String> health_checks = new ArrayList<>();
        ArrayList<String> check_dir = new ArrayList<>();
        ArrayList<String> create_dirs = new ArrayList<>();
        ArrayList<String> run_copy = new ArrayList<>();

        // Create list of commands for each machines
        assert hostnames != null;
        for (String hostname : hostnames) {
            health_checks.add("ssh -o StrictHostKeyChecking=no root@" + hostname + " hostname");
            create_dirs.add("ssh root@" + hostname + " if test ! -d " + remote_path + "; then mkdir -p " + remote_path + "; fi");
            check_dir.add("ssh root@" + hostname + " ls /tmp/root");
            run_copy.add("scp -r " + host_path + " root@" + hostname + ":" + remote_path);
        }

        // Apply health checker
        List<Boolean> returnValue = Utils.launch_actions_with_return(health_checks);

        // Check all machine are alive and deploy jar
        boolean isNodesOk = returnValue.stream().allMatch(x -> x);
        if (isNodesOk) {
            Utils.launch_actions_without_return(create_dirs);

            // wait for directories creation and check the creation
            Thread.sleep(3000);
            returnValue = Utils.launch_actions_with_return(check_dir);

            // if all directories are created, deploy jar file
            if (returnValue.stream().allMatch(x -> x)) {
                Utils.launch_actions_without_return(run_copy);
            } else {
                System.out.println("Something went wrong during the directories creation");
                return false;
            }
        } else {
            System.out.println("Not all nodes are safe, please check connection");
            return false;
        }
        return true;
    }

    public static void main(String[] args) throws InterruptedException {
        System.out.println("Hello from deploy!");
    }
}

In [32]:
ArrayList<String> hostnames = Utils.read_file("data/hostnames.txt");
String path_to_jar = "jar/job.jar";
String path_to_remote_host = "/tmp/root/";

Deploy.deploy(hostnames,
          path_to_jar,
          path_to_remote_host);

worker-2
worker-3
worker-1
ssh root@worker-1 if test ! -d /tmp/root/; then mkdir -p /tmp/root/; fi
ssh root@worker-2 if test ! -d /tmp/root/; then mkdir -p /tmp/root/; fi
ssh root@worker-3 if test ! -d /tmp/root/; then mkdir -p /tmp/root/; fi
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
job.jar
job.jar
job.jar
scp -r jar/job.jar root@worker-1:/tmp/root/
scp -r jar/job.jar root@worker-2:/tmp/root/
scp -r jar/job.jar root@worker-3:/tmp/root/
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


true

### <font color="red">Etape 8</font> : nettoyer un ensemble de machines avec CLEAN.

__1. Un programme “CLEAN” qui nettoie les machines distantes.__

Créez un nouveau programme CLEAN (différent de MASTER, SLAVE ou DEPLOY) qui efface votre dossier /tmp/<votre nom d’utilisateur>/sur les ordinateurs dont la connection SSH  fonctionne. Pour cela, vous utiliserez le même fichier texte écrit à la main et utilisé par DEPLOY contenant : les adresses IP et/ou les noms des machines que nous voulons utiliser pour notre système réparti (par exemple toutes les machines de cette salle de TP), avec un nom ou une IP par ligne dans le fichier.

CLEAN lit ce fichier ligne par ligne et efface sur chacune des machines votre dossier créé dans le dossier temporaire, en lançant la commande à distance 
```sh
rm -rf /tmp/<votre nom d’utilisateur>/
```
Attention de bien attendre la fin de la commande `rm -rf` pour être sûr que l’effacement a bien été effectué. 

Votre programme CLEAN lance-t-il les commande d’effacement de manière séquentielle (les unes après les autres) ou de manière parallèle?

__2. Vérification du DEPLOY et du CLEAN__

Vérifiez manuellement que l’effacement des dossiers a bien lieu. Ce programme vous permet de nettoyer un ensemble de machines avant de relancer un calcul. A partir de maintenant, vous pouvez déployer votre application en utilisant DEPLOY et vous pouvez nettoyer votre application en utilisant CLEAN. Vérifiez donc que DEPLOY suivi CLEAN fonctionne correctement.

Les lancements des commandes sur les machines sont fait en parallèles via les méthodes de la classe `Utils` :

- `launch_actions_with_return` : Lance un processus et récupère la sortie.
- `launch_actions_without_return` : Lance un processus sans attendre de retour.

La méthode `parallelStream()` permet de parcourir et de lancer chaque commande de notre liste en parallèle.

In [3]:
import java.util.ArrayList;
import java.util.List;

public class Clean {
    public static void main(String[] args) throws InterruptedException {
        ArrayList<String> hostnames = Utils.read_file(args[0]);
        ArrayList<String> remove_folder = new ArrayList<>();
        ArrayList<String> check_remove = new ArrayList<>();

        // Create list of commands for each machines
        assert hostnames != null;
        for (String hostname : hostnames){
            remove_folder.add("ssh root@" + hostname + " rm -rf /tmp/root");
            check_remove.add("ssh root@" + hostname + " ls /tmp/root");
        }

        // Apply health checker
        List<Boolean> returnValue = Utils.launch_actions_with_return(remove_folder);
        System.out.println(returnValue);
        returnValue = Utils.launch_actions_with_return(check_remove);
        System.out.println(returnValue);
    }
}

In [29]:
String[] args = {"data/hostnames.txt"};

Clean.main(args);

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
[true, true, true]
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
ls: cannot access '/tmp/root': No such file or directory
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
ls: cannot access '/tmp/root': No such file or directory
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
ls: cannot access '/tmp/root': No such file or directory
[false, false, false]


### <font color="red">Etape 10-12-13</font> : MapReduce - SPLIT, MAP, SHUFFLE et REDUCE.

__Activer le kernel python3__

La cellule suivante créer les fichiers nécessaire pour notre mapreduce

In [1]:
!mkdir -p /tmp/root/splits
!echo "Deer Beer River" > /tmp/root/splits/S0.txt
!echo "Car Car River" > /tmp/root/splits/S1.txt
!echo "Deer Car Beer" > /tmp/root/splits/S2.txt

__Activer le kernel java et recompiler les classes (Deploy, Clean et Utils)__

La classe `Master` ci-dessous lance les différentes phase du mapreduce en chronométrant chaque étape. Le `Master` attend la fin de chaque étape via la méthode `join` de la classe `Thread` en Java. Le `Master` attend que chaque processus lancé avec les méthodes `launch_actions_with_return` et `launch_actions_without_return` se termine avant de poursuivre.

In [5]:
import java.util.*;

public class Master {
    public static void main(String[] args) throws InterruptedException {
        ArrayList<String> hostnames = Utils.read_file(args[0]);
        Set<String> slaves = new HashSet<>();
        List<String> files = Utils.list_directory(args[1]);
        ArrayList<String> health_checks = new ArrayList<>();
        ArrayList<String> create_dirs = new ArrayList<>();
        ArrayList<String> check_dir = new ArrayList<>();
        ArrayList<String> run_copy = new ArrayList<>();
        ArrayList<String> run_map = new ArrayList<>();
        ArrayList<String> run_shuffle = new ArrayList<>();
        ArrayList<String> run_reduce = new ArrayList<>();
        ArrayList<String> copy_hostnames_file = new ArrayList<>();
        Random rand = new Random();

        for (String file:files){
            assert hostnames != null;
            String slave = hostnames.get(rand.nextInt(hostnames.size()));
            slaves.add(slave);
            health_checks.add("ssh -o StrictHostKeyChecking=no root@" + slave + " hostname");
            create_dirs.add("ssh root@" + slave + " if test ! -d " + args[2] + "; then mkdir -p " + args[2] + "; fi");
            run_copy.add("scp -r " + file + " root@" + slave + ":" + args[2]);
            check_dir.add("ssh root@" + slave + " ls " + args[2]);
            run_map.add("ssh root@" + slave + " java -jar /tmp/root/job.jar 0 " + file);
            copy_hostnames_file.add("scp " + args[0] + " root@" + slave + ":/tmp/root");

        }
        for (String slave:slaves)
            run_shuffle.add("ssh root@" + slave + " java -jar /tmp/root/job.jar 1 ");

        assert hostnames != null;
        for (String slave:hostnames)
            run_reduce.add("ssh root@" + slave + " java -jar /tmp/root/job.jar 2 ");

        // Apply health checker
        List<Boolean> returnValue = Utils.launch_actions_with_return(health_checks);

        // Check all machine are alive and deploy jar
        boolean isNodesOk = returnValue.stream().allMatch(x -> x);
        if (isNodesOk) {
            Utils.launch_actions_without_return(create_dirs);

            // wait for directories creation and check the creation
            Thread.sleep(3000);
            returnValue = Utils.launch_actions_with_return(check_dir);

            // if all directories are created, deploy jar file
            if (returnValue.stream().allMatch(x -> x)) {
                Utils.launch_actions_without_return(run_copy);
            } else {
                System.out.println("Something went wrong during the directories creation");
            }
        } else {
            System.out.println("Not all nodes are safe, please check connection");
        }

        // Map
        double start_map_time = System.currentTimeMillis();
        Deploy.deploy(hostnames,
                "jar/job.jar", "/tmp/root/");
        Utils.launch_actions_without_return(run_map);
        System.out.println("MAP FINISHED");
        double end_map_time = System.currentTimeMillis();
        double total_map_time = end_map_time - start_map_time;
        System.out.println(String.format("map time: %f", total_map_time/1000));

        // shuffle
        double start_shuffle_time = System.currentTimeMillis();
        Utils.launch_actions_without_return(copy_hostnames_file);
        Utils.launch_actions_without_return(run_shuffle);
        System.out.println("SHUFFLE FINISHED");
        double end_shuffle_time = System.currentTimeMillis();
        double total_shuffle_time = end_shuffle_time - start_shuffle_time;
        System.out.println(String.format("shuffle time: %f", total_shuffle_time/1000));

        // reduce
        double start_reduce_time = System.currentTimeMillis();
        Utils.launch_actions_without_return(run_reduce);
        System.out.println("REDUCE FINISHED");
        double end_reduce_time = System.currentTimeMillis();
        double total_reduce_time = end_reduce_time - start_reduce_time;
        System.out.println(String.format("reduce time: %f", total_reduce_time/1000));
    }
}

Ci-dessous la classe `Slave` utilisé pour lancer les différentes étapes du mapreduce. Le jar job.jar est générée à partir de cette classe.

In [27]:
import java.io.File;
import java.io.IOException;
import java.util.*;
import java.util.Map.Entry;
import java.net.InetAddress;

import static java.util.stream.Collectors.*;

public class Slave{

    private static ArrayList<String> transform_key_value(String filename){
        // Function for the question 1
        ArrayList<String> lines = Utils.read_file(filename);
        assert lines != null;
        ArrayList<String[]> lines_tokenized = Utils.tokenize(lines);
        ArrayList<String> words_count = new ArrayList<>();

        for (String[] line_tokenized: lines_tokenized) {
            for (String word:line_tokenized) {
                if (!word.isEmpty()){
                    words_count.add(word + " 1.0");
                }
            }
        }
        return words_count;
    }

    private static HashMap<String, Double> words_count(String filename){
        // Function for the question 1
        ArrayList<String> lines = Utils.read_file(filename);
        assert lines != null;
        ArrayList<String[]> lines_tokenized = Utils.tokenize(lines);
        HashMap<String, Double> words_count = new HashMap<>();

        for (String[] line_tokenized: lines_tokenized) {
            String word = line_tokenized[0];
            if (!word.isEmpty()){
                if (words_count.get(word) == null) {
                    words_count.put(word, 1.0);
                } else {
                    words_count.put(word, words_count.get(word)+1);
                }
            }
        }
        return words_count;
    }

    public static HashMap<String, Double> sorted_map_by_numeric_value(HashMap<String, Double> hash_map){
        // Function for question 2
        return hash_map
                .entrySet()
                .stream()
                .sorted(Collections.reverseOrder(Entry.comparingByValue()))
                .collect(toMap(Entry::getKey, Entry::getValue, (e1, e2) -> e2,
                        LinkedHashMap::new));
    }

    public static HashMap<String, Double> sort(HashMap<String, Double> hash_map){
        // Function for question 3
        return hash_map
                .entrySet()
                .stream()
                .sorted(new Comparator<Entry<String, Double>>() {
                    @Override
                    public int compare(Entry<String, Double> e1, Entry<String, Double> e2) {
                        if (e1.getValue().equals(e2.getValue())) {
                            return e1.getKey().compareTo(e2.getKey());
                        } else {
                            return e2.getValue().compareTo(e1.getValue());
                        }
                    }
                }).collect(toMap(Entry::getKey, Entry::getValue, (e1, e2) -> e1,
                        LinkedHashMap::new));
    }

    public static void print_map(Map<String, Double> hash_map) {
        for (Entry<String, Double> el:hash_map.entrySet()) {
            System.out.println(el.getKey() + " " + el.getValue());
        }
    }

    public static void print_map(Map<String, Double> hash_map, Integer n) {
        Integer count = 0;
        for (Entry<String, Double> el:hash_map.entrySet()) {
            if(count.equals(n)){
                break;
            } else {
                System.out.println(el.getKey() + " " + el.getValue());
                count++;
            }
        }
    }

    private static void map(String src_file, String map_file) throws IOException {
        ArrayList<String> words_count = Slave.transform_key_value(src_file);
        Utils.write_file(words_count, map_file, "a");
    }

    private static void shuffle(String file, ArrayList<String> hostnames) throws IOException, InterruptedException {
        String current_host = InetAddress.getLocalHost().getHostName();
        int nb_slaves = hostnames.size();

        ArrayList<String[]> keys_values = Utils.tokenize(Objects.requireNonNull(Utils.read_file(file)));
        for (String[] key_value:keys_values){
            int hash_key = key_value[0].hashCode();
            int compute_slave = hash_key%nb_slaves;
            if (!hostnames.get(compute_slave).equals(current_host)){
                File shuffle_file = new File ("/tmp/root/shuffle" + "/" + String.format("%d_%d_%d.txt",
                                                                                            hash_key,
                                                                                            compute_slave,
                                                                                            current_host.hashCode()));
                Utils.write_file(Collections.singletonList(key_value[0] + " " + key_value[1]),
                        shuffle_file.toString(),
                        "a");
                Deploy.deploy(Collections.singletonList(hostnames.get(compute_slave)),
                        shuffle_file.toString(), "/tmp/root/shuffle");
                shuffle_file.delete();
            } else {
                File shuffle_file = new File ("/tmp/root/shuffle" + "/" + hash_key + "_" + compute_slave + ".txt");
                Utils.write_file(Collections.singletonList(key_value[0] + " " + key_value[1]),
                         shuffle_file.toString(),
                        "a");
            }
        }
    }

    private static void reduce(String reduce_directory, String shuffle_directory) throws IOException {
        List<String> files = Utils.list_directory(shuffle_directory);
        HashMap<String, Double> words_count = new HashMap<>();

        for (String file:files) {
            words_count(file).forEach(
                    (key, value) -> words_count.merge(key, value, Double::sum)
            );
        }
        for (Entry<String, Double> key_value:words_count.entrySet()){
            int hash = key_value.getKey().hashCode();
            File reduce_file = new File(reduce_directory + "/" + hash + ".txt");
            Utils.write_file(Collections.singletonList(key_value.getKey() + " " + key_value.getValue()),
                                                    reduce_file.toString(), "w");
        }
    }

    public static void main(String[] args) throws IOException, InterruptedException {
        if (args[0].equals("0")){
            File split_file = new File(args[1]);
            String num = args[1].replaceAll("[^\\d]", "");
            File map_directory = new File(new File(split_file.getParent()).getParent() + "/maps");
            if (!map_directory.exists()){
                boolean result = map_directory.mkdir();
                if (!result){
                    System.out.println("Something goes wrong when creating maps folder");
                } else {
                    map(split_file.toString(), map_directory + "/UM" + num + ".txt");
                }
            } else {
                map(split_file.toString(), map_directory + "/UM" + num + ".txt");
            }
        } else if (args[0].equals("1")){
            ArrayList<String> hostnames = Utils.read_file("/tmp/root/hostnames.txt");
            ArrayList<String> health_checks = new ArrayList<>();
            assert hostnames != null;
            for (String hostname : hostnames)
                health_checks.add("ssh -o StrictHostKeyChecking=no root@" + hostname + " hostname");
            File shuffle_directory = new File("/tmp/root/shuffle");
            if (!shuffle_directory.exists()){
                boolean result = shuffle_directory.mkdir();
                if (!result){
                    System.out.println("Something goes wrong when creating shuffle folder");
                } else {
                    for (String file: Utils.list_directory("/tmp/root/maps")){
                        shuffle(file, hostnames);
                    }
                }
            } else {
                for (String file: Utils.list_directory("/tmp/root/maps")){
                    shuffle(file, hostnames);
                }
            }
        } else if (args[0].equals("2")){
            File reduce_directory = new File("/tmp/root/reduce");
            File shuffle_directory = new File("/tmp/root/shuffle");
            if (!reduce_directory.exists()){
                boolean result = reduce_directory.mkdir();
                if (!result){
                    System.out.println("Something goes wrong when creating reduce folder");
                } else {
                    reduce(reduce_directory.toString(), shuffle_directory.toString());
                }
            } else {
                reduce(reduce_directory.toString(), shuffle_directory.toString());
            }
        }

    }
}

In [30]:
String[] args = {"data/hostnames.txt", "/tmp/root/splits", "/tmp/root/splits"};

Master.main(args);

worker-1
worker-2
worker-2
ssh root@worker-1 if test ! -d /tmp/root/splits; then mkdir -p /tmp/root/splits; fi
ssh root@worker-2 if test ! -d /tmp/root/splits; then mkdir -p /tmp/root/splits; fi
ssh root@worker-2 if test ! -d /tmp/root/splits; then mkdir -p /tmp/root/splits; fi
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
scp -r /tmp/root/splits/S0.txt root@worker-1:/tmp/root/splits
scp -r /tmp/root/splits/S1.txt root@worker-2:/tmp/root/splits
scp -r /tmp/root/splits/S2.txt root@worker-2:/tmp/root/splits
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change loca

Ci-dessous le temps de calcul pour charque étape:

- map : 3.988000 secondes
- shuffle : 11.685000 secondes
- reduce : 0.27700 secondes

Bien évidemment, la partie shuffle prend le plus de temps. C'est normal au vu de tous les transfets réseau nécessaire pour envoyer les clés sur les bons noeuds.

__Activer le kernel python3__

La cellule ci-dessous afficher les fichiers présents dans le dossier reduce de chaque worker. Le résultat attendu est donc :

- beer 2
- river 2
- car 3
- deer 2

In [4]:
!echo worker-1 && ssh root@worker-1 find /tmp/root/reduce -type f -exec cat {} +
!echo worker-2 && ssh root@worker-2 find /tmp/root/reduce -type f -exec cat {} +
!echo worker-3 && ssh root@worker-3 find /tmp/root/reduce -type f -exec cat {} +

worker-1
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
beer 2.0
river 2.0
worker-2
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
car 3.0
worker-3
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
deer 2.0
